## EXTRACION DE DATOS

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

c:\Users\SPARTAN PC\miniconda3\envs\langchain_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Resuming download from 621805568 bytes (1841559867 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/paultimothymooney/chest-xray-pneumonia?dataset_version_number=2 (621805568/2463365435) bytes left.


100%|██████████| 2.29G/2.29G [03:27<00:00, 8.87MB/s]

Extracting files...


Path to dataset files: C:\Users\SPARTAN PC\.cache\kagglehub\datasets\paultimothymooney\chest-xray-pneumonia\versions\2


## ESTRUCTURA DE LOS DIRECTORIOS

In [4]:
import os
for root, dirs, files in os.walk(path):
    level = root.replace(path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:3]:  # Mostrar solo los primeros 3 archivos
        print(f'{subindent}{file}')
    if len(files) > 3:
        print(f'{subindent}... y {len(files) - 3} archivos más')


2/
  chest_xray/
    chest_xray/
      .DS_Store
      test/
        .DS_Store
        NORMAL/
          IM-0001-0001.jpeg
          IM-0003-0001.jpeg
          IM-0005-0001.jpeg
          ... y 231 archivos más
        PNEUMONIA/
          person100_bacteria_475.jpeg
          person100_bacteria_477.jpeg
          person100_bacteria_478.jpeg
          ... y 387 archivos más
      train/
        .DS_Store
        NORMAL/
          .DS_Store
          IM-0115-0001.jpeg
          IM-0117-0001.jpeg
          ... y 1339 archivos más
        PNEUMONIA/
          .DS_Store
          person1000_bacteria_2931.jpeg
          person1000_virus_1681.jpeg
          ... y 3873 archivos más
      val/
        .DS_Store
        NORMAL/
          .DS_Store
          NORMAL2-IM-1427-0001.jpeg
          NORMAL2-IM-1430-0001.jpeg
          ... y 6 archivos más
        PNEUMONIA/
          .DS_Store
          person1946_bacteria_4874.jpeg
          person1946_bacteria_4875.jpeg
          ... y 6 archivos m

In [13]:
path = path + "/chest_xray"  # Ajusta según la estructura del dataset

### SEPARAMOS Y ANALISAMOS LA DISTRIBUCION DE IMAGENES

In [14]:
train_path = os.path.join(path, 'train')
test_path = os.path.join(path, 'test')
val_path = os.path.join(path, 'val')

class_counts = {}
for split in ['train', 'test', 'val']:
    split_path = os.path.join(path, split)
    class_counts[split] = {}
    for class_name in os.listdir(split_path):
        class_path = os.path.join(split_path, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.endswith(('.png', '.jpg', '.jpeg'))])
            class_counts[split][class_name] = count

print("\n" + "=" * 50)
print("DISTRIBUCIÓN DE CLASES")
print("=" * 50)
for split, classes in class_counts.items():
    print(f"\n{split.upper()}:")
    for class_name, count in classes.items():
        print(f"  {class_name}: {count} imágenes")


DISTRIBUCIÓN DE CLASES

TRAIN:
  NORMAL: 1341 imágenes
  PNEUMONIA: 3875 imágenes

TEST:
  NORMAL: 234 imágenes
  PNEUMONIA: 390 imágenes

VAL:
  NORMAL: 8 imágenes
  PNEUMONIA: 8 imágenes


#### El set de training esta desbalanceado, ya que son pocas imagenes vamos a aplicar tecnicas que ayuden a disminuir el gap, ya que actualmente por cada normal hay 2.9 iamagenes de pneumonia.
#### Debido a lo anterior he decidido aplicar data augmentation en el dataset de training, ya que al contar con tan pocas imagenes  al menos del lado de Normal puede causar overfitting y tambien tome la desicion de aplciar Ponderación de Clases para evitar disminuir el dataset y tener un entrenamiento mas equilibrado ya que a pesar de aplicar data augmentantion el dataset sigue con el mismo sesgo al aplicarlo a todo el dataset (ambas clasess)
#### 1. combinacion de undersampling y oversampling para balancear la clase 
#### 2. Ponderacion de clases